In [ ]:
# Common Opponent Lister - for any team (myhockeyrankings team id needed), go through all their opponents
# looking for common opponents with Xtreme. Print out a list of common opponents and results for both Xtreme
# and upcoming opponent. 


In [ ]:
# install pre-reqs
!pip install pycurl
!pip install beautifulsoup4

# imports 
import pycurl
import pandas as pd 
from bs4 import BeautifulSoup
from io import BytesIO
from google.colab import files


     |████████████████████████████████| 227 kB 7.8 MB/s 
  Created wheel for pycurl: filename=pycurl-7.44.1-cp37-cp37m-linux_x86_64.whl size=292986 sha256=194d036f684ec7085ce5e3b2058d06467711b56eafa88cfae771f9d82fbd30f2
  Stored in directory: /root/.cache/pip/wheels/c5/46/b4/4dc60b406282c22dd4f1ca7da5c949e88aeadb78283123f94d
Successfully built pycurl


In [ ]:
# function to build dataframe with all games
def get_games(id, yr):

  # grab the page for team 
  buffer = BytesIO()
  c = pycurl.Curl()
  c.setopt(c.URL, "https://myhockeyrankings.com/team_info.php?y=" + str(yr) + "&t=" + str(id))
  c.setopt(c.WRITEDATA, buffer)
  c.perform()
  c.close()

  # decode into a string we can work with 
  body = buffer.getvalue()
  html = body.decode('iso-8859-1')

  # pass that into beautiful soup 
  soup = BeautifulSoup(html)
  # extract the table will all the results 
  table = soup.find(id="team-scores")

  # loop through each row and add to dataframe
  output_rows = []
  for table_row in table.findAll('tr'):
      columns = table_row.findAll('td')
      output_row = []
      for column in columns:
          output_row.append(column.text)
      output_rows.append(output_row)
      #print(output_row)

  # also, grab the name of the team 
  team_name = soup.find(class_="sitestyle6").text
  #team_name = element.get_text()

  return output_rows, team_name


In [ ]:
# get the wlc09 game list 
our_team_id = "1114"
rows, team_name = get_games(our_team_id, "2021")
df_us = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Result','NA'])


In [ ]:
# get the opponent
opponent_id = "16650" # 
rows, team_name = get_games(opponent_id, "2021")
df_opponent = pd.DataFrame(rows, columns = ['Date', 'Time', 'Opponent','Location','Result','NA'])


In [ ]:
# shorten down to unique opponents
opp_list  = df_us.Opponent.unique()

df_common = df_opponent[df_opponent.Opponent.isin(opp_list)]

# shut off warning
pd.options.mode.chained_assignment = None

# clean up the dataframe by dropping some colums
del df_common["Time"] # don't care 
del df_common["Location"] # don't care 
del df_common["NA"] # don't care
df_common.drop(df_common.head(1).index, inplace=True) # first row always says none for some reason

# remove any spaces in the team name
team_name.replace(" ", "")

# save as csv
df_common.to_csv("/content/" + team_name + "_common_opponents.csv", index=False)

# and display it
df_common

,Date,Opponent,Result
1,2021-09-18,New Jersey Colonials 09 AAA,3 - 3
3,2021-09-25,Philadelphia Jr Flyers 09 AAA,2 - 8
4,2021-09-26,Long Island Royals 09 AAA,1 - 8
5,2021-10-02,New Jersey Devils 09 AAA,4 - 11
9,2021-11-20,New Jersey Devils 09 AAA,-
10,2021-12-11,New York Saints 09 AAA,-
11,2021-12-11,New York Saints 09 AAA,-
12,2021-12-12,New Jersey Devils 09 AAA,-
13,2022-01-29,New Jersey Devils 09 AAA,-
14,2022-01-30,New York Saints 09 AAA,-
